## DescendantCombinator

```css
div.book p.price
```

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, DescendantCombinator, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <p class="price">Price: $20</p>
        </div>
    """,
    features="lxml",
)

book_selector = TagSelector(
    "div", attributes=[AttributeSelector("class", value="book")]
)
price_selector = TagSelector(
    "p", attributes=[AttributeSelector("class", value="price")]
)
selector = DescendantCombinator(book_selector, price_selector)
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <p class="price">Price: $20</p>
        </div>
    """,
    features="lxml",
)

book_selector = TagSelector(
    "div", attributes=[AttributeSelector("class", value="book")]
)
price_selector = TagSelector(
    "p", attributes=[AttributeSelector("class", value="price")]
)
selector = book_selector >> price_selector
selector.find(soup)

In [ ]:
from soupsavvy import AttributeSelector, DescendantCombinator, TagSelector

book_selector = TagSelector(
    "div", attributes=[AttributeSelector("class", value="book")]
)
price_selector = TagSelector(
    "p", attributes=[AttributeSelector("class", value="price")]
)
print(
    book_selector >> price_selector
    == DescendantCombinator(book_selector, price_selector)
)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, IdSelector, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <p class="price">Price: $10</p>
        </div>
        <div id="available">
            <div class="book">
                <span class="title">Animal Farm</span>
                <p class="price">Price: $20</p>
            </div>
        </div>
    """,
    features="lxml",
)

available_selector = TagSelector("div", attributes=[IdSelector("available")])
book_selector = TagSelector("div", attributes=[ClassSelector("book")])
price_selector = ClassSelector("price")
selector = available_selector >> book_selector >> price_selector
selector.find(soup)

In [ ]:
from soupsavvy import ClassSelector, DescendantCombinator, IdSelector, TagSelector

available_selector = TagSelector("div", attributes=[IdSelector("available")])
book_selector = TagSelector("div", attributes=[ClassSelector("book")])
price_selector = ClassSelector("price")
selector = available_selector >> book_selector >> price_selector

print(
    selector == DescendantCombinator(available_selector, book_selector, price_selector)
)
print(
    selector == DescendantCombinator(book_selector, available_selector, price_selector)
)

## ChildCombinator

```css
div.book > p.price
```

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, ChildCombinator, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="discount">
                <h2>Discounted</h2>
                <p class="price">Price: $15</p>
            </span>
            <p class="price">Price: $20</p>
        </div>
    """,
    features="lxml",
)

book_selector = TagSelector(
    "div", attributes=[AttributeSelector("class", value="book")]
)
price_selector = TagSelector(
    "p", attributes=[AttributeSelector("class", value="price")]
)
selector = ChildCombinator(book_selector, price_selector)
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="discount">
                <h2>Discounted</h2>
                <p class="price">Price: $15</p>
            </span>
            <p class="price">Price: $20</p>
        </div>
    """,
    features="lxml",
)

book_selector = TagSelector("div", attributes=[ClassSelector("book")])
price_selector = TagSelector("p", attributes=[ClassSelector("price")])
selector = book_selector > price_selector
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, IdSelector, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <p class="price">Price: $10</p>
        </div>
        <div id="available">
            <div class="book">
                <span class="title">Animal Farm</span>
                <span class="discount">
                    <h2>Discounted</h2>
                    <p class="price">Price: $15</p>
                </span>
                <p class="price">Price: $20</p>
            </div>
        </div>
    """,
    features="lxml",
)

available_selector = TagSelector("div", attributes=[IdSelector("available")])
book_selector = TagSelector("div", attributes=[ClassSelector("book")])
price_selector = ClassSelector("price")
selector = available_selector > book_selector >> price_selector
selector.find(soup)

In [ ]:
from soupsavvy import ClassSelector, DescendantCombinator, IdSelector, TagSelector

available_selector = TagSelector("div", attributes=[IdSelector("available")])
book_selector = TagSelector("div", attributes=[ClassSelector("book")])
price_selector = ClassSelector("price")
  
print(
    ChildCombinator(
        available_selector, DescendantCombinator(book_selector, price_selector)
    )
    == available_selector > book_selector >> price_selector
)

## NextSiblingCombinator

```css
div.book + p.price
```

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, NextSiblingCombinator, PatternSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <span class="discount">
            <h2>Discounted</h2>
            <p class="price">Price: $15</p>
            <h2>Highest Price</h2>
            <p class="price">Price: $20</p>
        </span>
        <p class="price">Price: $20</p>
    """,
    features="lxml",
)

discount_selector = PatternSelector("Discounted")
price_selector = ClassSelector("price")
selector = NextSiblingCombinator(discount_selector, price_selector)
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, PatternSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <span class="discount">
            <h2>Discounted</h2>
            <p class="price">Price: $15</p>
            <h2>Highest Price</h2>
            <p class="price">Price: $20</p>
        </span>
        <p class="price">Price: $30</p>
    """,
    features="lxml",
)

discount_selector = PatternSelector("Discounted")
price_selector = ClassSelector("price")
selector = discount_selector + price_selector
selector.find(soup)

## SubsequentSiblingCombinator

```css
div.book ~ p.price
```

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    SubsequentSiblingCombinator,
    TagSelector,
)

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <span class="discount">
            <p class="price">Price: $25</p>
            <h2>Discounted</h2>
            <span>Bargain!!!</span>
            <p class="price">Price: $15</p>
        </span>
        <p class="price">Price: $20</p>
    """,
    features="lxml",
)

discount_selector = TagSelector("h2")
price_selector = ClassSelector("price")
selector = SubsequentSiblingCombinator(discount_selector, price_selector)
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, TagSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <span class="discount">
            <p class="price">Price: $25</p>
            <h2>Discounted</h2>
            <span>Bargain!!!</span>
            <p class="price">Price: $15</p>
        </span>
        <p class="price">Price: $20</p>
    """,
    features="lxml",
)

discount_selector = TagSelector("h2")
price_selector = ClassSelector("price")
selector = discount_selector * price_selector
selector.find(soup)

## AndSelector

```css
p.price#best
```

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    AndSelector,
    ClassSelector,
    IdSelector,
    PatternSelector,
    TagSelector,
)

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="price">Price: $30</p>
        <p class="price">Price: $20</p>
        <p class="price" id="best">Price: €12</p>
        <p class="price" id="best">Price: $15</p>
    """,
    features="lxml",
)

price_selector = TagSelector(
    "p", attributes=[ClassSelector("price"), IdSelector("best")]
)
dollars_selector = PatternSelector(re.compile(r"\$\d+"))
selector = AndSelector(price_selector, dollars_selector)
selector.find(soup)

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    IdSelector,
    PatternSelector,
    TagSelector,
)

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="price">Price: $30</p>
        <p class="price">Price: $20</p>
        <p class="price" id="best">Price: €12</p>
        <p class="price" id="best">Price: $15</p>
    """,
    features="lxml",
)

selector = (
    TagSelector("p")
    & ClassSelector("price")
    & IdSelector("best")
    & PatternSelector(re.compile(r"\$\d+"))
)
selector.find(soup)

## OrSelector (SelectorList)

```css
h1, h1 { color: red;}
:is(h1, h2) { color: red; }
```


In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    OrSelector,
    PatternSelector,
)

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <span class="discount">
            <h2>Discounted</h2>
            <p class="price">Price: $25</p>
            <p class="price"><s>Price: $30</s></p>
            <p class="price"><s>Price: $35</s></p>
        </span>
        <p class="title">Brave New World</p>
        <p class="price">Price: $15</p>
    """,
    features="lxml",
)

discount_selector = ClassSelector("discount") > PatternSelector(
    "Discounted"
) + ClassSelector("price")
standard_price_selector = ClassSelector("title") + PatternSelector(re.compile(r"\$\d+"))
selector = OrSelector(discount_selector, standard_price_selector)
selector.find_all(soup)

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    OrSelector,
    PatternSelector,
)

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <span class="discount">
            <h2>Discounted</h2>
            <p class="price">Price: $25</p>
            <p class="price"><s>Price: $30</s></p>
            <p class="price"><s>Price: $35</s></p>
        </span>
        <p class="title">Brave New World</p>
        <p class="price">Price: $15</p>
    """,
    features="lxml",
)

discount_selector = ClassSelector("discount") > PatternSelector(
    "Discounted"
) + ClassSelector("price")
standard_price_selector = ClassSelector("title") + PatternSelector(re.compile(r"\$\d+"))
selector = discount_selector | standard_price_selector
selector.find_all(soup)

In [ ]:
from soupsavvy import ClassSelector, OrSelector, SelectorList

discount_selector = ClassSelector("discount")
price_selector = ClassSelector("price")

print(
    SelectorList(discount_selector, price_selector)
    == discount_selector | price_selector
    == OrSelector(discount_selector, price_selector)
)

In [ ]:
from soupsavvy import ClassSelector, SelectorList

discount_selector = ClassSelector("discount")
price_selector = ClassSelector("price")

print(
    SelectorList(discount_selector, price_selector)
    == SelectorList(price_selector, discount_selector)
)

## NotSelector

```css	
p:not(.price)
```

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, NotSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = NotSelector(discount_selector)
selector.find(soup)

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    NotSelector,
    PatternSelector,
)

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
dollars_selector = PatternSelector(re.compile(r"\$\d+"))
selector = NotSelector(discount_selector, dollars_selector)
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = ~discount_selector
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = ~NotSelector(discount_selector)
print(selector)
selector.find(soup)

## HasSelector

```css
div:has(p)
```

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Brave New World</span>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Animal Farm</span>
        <span class="discount">
            <p class="price">Price: $15</p>
        </span>
        <p class="price">Price: $20</p>
    </div>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = HasSelector(discount_selector)
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector, PatternSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Brave New World</span>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Animal Farm</span>
        <span class="discount">
            <p class="price">Price: $15</p>
        </span>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span>Bestseller</span>
        <span class="title">Frankenstein</span>
        <p class="price">Price: $30</p>
    </div>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
bestseller_selector = PatternSelector("Bestseller")
selector = HasSelector(discount_selector, bestseller_selector)
selector.find_all(soup)

## NthOfSelector

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector
from soupsavvy.selectors.nth import NthOfSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <span>Bestseller</span>
        <p class="price">Price: €15</p>
        <p class="price">Price: €25</p>
        <p class="price">Price: €17</p>
    """,
    features="lxml",
)

price_selector = ClassSelector("price")
selector = NthOfSelector(price_selector, nth="2n")
selector.find_all(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector
from soupsavvy.selectors.nth import NthLastOfSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <span>Bestseller</span>
        <p class="price">Price: €15</p>
        <p class="price">Price: €25</p>
        <p class="price">Price: €17</p>
    """,
    features="lxml",
)

price_selector = ClassSelector("price")
selector = NthLastOfSelector(price_selector, nth="odd")
selector.find_all(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector
from soupsavvy.selectors.nth import OnlyOfSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Animal Farm</span>
        <p class="price">Price: $15</p>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Frankenstein</span>
        <p class="price">Price: $30</p>
    </div>
    """,
    features="html.parser",
)
price_selector = ClassSelector("price")
selector = HasSelector(OnlyOfSelector(price_selector))
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector
from soupsavvy.selectors.nth import OnlyOfSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Frankenstein</span>
        <p class="price">Price: $30</p>
    </div>
    <div class="book">
        <span class="title">Animal Farm</span>
        <p class="price">Price: $15</p>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Brave New world</span>
        <p class="price">Price: $25</p>
        <p class="price">Price: $20</p>
    </div>
    """,
    features="html.parser",
)
price_selector = ClassSelector("price")
selector = ~HasSelector(OnlyOfSelector(price_selector)) > ClassSelector("title")
selector.find_all(soup, recursive=False)

## Overview